<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 10. Бустинг
## <center> Часть 10. Продвинутые методы работы с категориальными признаками и CatBoost

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max.columns', 100)
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

Считаем данные и посмотрим на первые несколько строк. Видим, что у нас тут немало категориальных признаков.

In [2]:
df = pd.read_csv('../../data/bank.csv')

In [3]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
age          4521 non-null int64
job          4521 non-null object
marital      4521 non-null object
education    4521 non-null object
default      4521 non-null object
balance      4521 non-null int64
housing      4521 non-null object
loan         4521 non-null object
contact      4521 non-null object
day          4521 non-null int64
month        4521 non-null object
duration     4521 non-null int64
campaign     4521 non-null int64
pdays        4521 non-null int64
previous     4521 non-null int64
poutcome     4521 non-null object
y            4521 non-null int64
dtypes: int64(8), object(9)
memory usage: 600.5+ KB


Всего 9 признаков со строковыми значениями.

In [5]:
df.columns[df.dtypes == 'object']

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object')

## Без категориальных признаков
Попытаемся сначала просто проигнорировать категориальные признаки. Обучим случайный лес и посмотрим на ROC AUC на кросс-валидации и на отоженной выборке. Это будет наш бейзлайн. 

In [9]:
df_no_cat, y = df.loc[:, df.dtypes != 'object'].drop('y', axis=1), df['y']

In [10]:
df_no_cat_part, df_no_cat_valid, y_train_part, y_valid = train_test_split(df_no_cat, y,
                                                                            test_size=.3, 
                                                                            stratify=y,
                                                                            random_state=17)

In [11]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [12]:
forest = RandomForestClassifier(random_state=17)

In [14]:
np.mean(cross_val_score(forest, df_no_cat_part, y_train_part, cv=skf, scoring='roc_auc'));

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

In [15]:
forest.fit(df_no_cat_part, y_train_part)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

In [16]:
roc_auc_score(y_valid, forest.predict_proba(df_no_cat_valid)[:, 1])

0.8246600055509299

In [17]:
forest.feature_importances_

array([0.1473972 , 0.1775853 , 0.12930319, 0.35103475, 0.06646041,
       0.08351306, 0.0447061 ])

## LabelEncoder для категориальных признаков
Сделаем то же самое, но попробуем закодировать категориальные признаки по-простому: с помощью `LabelEncoder`.

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
label_encoder = LabelEncoder()

In [20]:
df_cat_label_enc = df.copy().drop('y', axis=1)
for col in df.columns[df.dtypes == 'object']:
    df_cat_label_enc[col] = label_encoder.fit_transform(df_cat_label_enc[col])

In [21]:
df_cat_label_enc.shape

(4521, 16)

In [22]:
df_cat_label_enc.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,30,10,1,0,0,1787,0,0,0,19,10,79,1,-1,0,3
1,33,7,1,1,0,4789,1,1,0,11,8,220,1,339,4,0
2,35,4,2,2,0,1350,1,0,0,16,0,185,1,330,1,0
3,30,4,1,2,0,1476,1,1,2,3,6,199,4,-1,0,3
4,59,1,1,1,0,0,1,0,2,5,8,226,1,-1,0,3


In [17]:
df_cat_label_enc_part, df_cat_label_enc_valid = train_test_split(df_cat_label_enc, test_size=.3, 
                                                    stratify=y, random_state=17)

In [18]:
np.mean(cross_val_score(forest, df_cat_label_enc_part, y_train_part, cv=skf, scoring='roc_auc'))

0.84296976359098053

In [19]:
forest.fit(df_cat_label_enc_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [20]:
roc_auc_score(y_valid, forest.predict_proba(df_cat_label_enc_valid)[:, 1])

0.84814737718567856

## Бинаризация категориальных признаков (dummies, OHE)
Теперь сделаем то, что обычно по умолчанию и делают – бинаризацию категориальных признаков. Dummy-признаки, One-Hot Encoding... с небольшими различиями это об одном же - для каждого значения каждого категориального признака завести свой бинарный признак.

In [23]:
df_cat_dummies = pd.get_dummies(df, columns=df.columns[df.dtypes == 'object']).drop('y', axis=1)

In [24]:
df_cat_dummies.shape

(4521, 51)

In [26]:
df_cat_dummies.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,30,1787,19,79,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,33,4789,11,220,1,339,4,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,35,1350,16,185,1,330,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,30,1476,3,199,4,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
4,59,0,5,226,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [27]:
df_cat_dummies_part, df_cat_dummies_valid = train_test_split(df_cat_dummies, test_size=.3, 
                                                    stratify=y, random_state=17)

In [28]:
np.mean(cross_val_score(forest, df_cat_dummies_part, y_train_part, cv=skf, scoring='roc_auc'))

0.84579530735273

In [29]:
forest.fit(df_cat_dummies_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

In [40]:
roc_auc_score(y_valid, forest.predict_proba(df_cat_dummies_valid)[:, 1])

0.8587742052563034

In [38]:
forest.predict_proba(df_cat_dummies_valid)[:,1]

array([0. , 0.2, 0. , ..., 0.4, 0.1, 0. ])

## Попарные взаимодействия признаков
Пока лес все еще лучше регрессии (хотя мы не тюнили гиперпараметры, но и не будем). Мы хотим идти дальше. Мощной техникой для работы с категориальными признаками будет учет попарных взаимодействий признаков (feature interactions). Построим попарные взаимодействия всех признаков. Вообще тут можно пойти дальше и строить взаимодействия трех и более признаков. Owen Zhang [как-то строил](https://www.youtube.com/watch?v=LgLcfZjNF44) даже 7-way interactions. Чего не сделаешь ради победы на Kaggle! :)

In [41]:
df_interact = df.copy()

In [43]:
cat_features = df.columns[df.dtypes == 'object']
for i, col1 in enumerate(cat_features):
    print(i, col1)
    for j, col2 in enumerate(cat_features[i + 1:]):
        print('WITH',j, col2)
        df_interact[col1 + '_' + col2] = df_interact[col1] + '_' + df_interact[col2] 

0 job
WITH 0 marital
WITH 1 education
WITH 2 default
WITH 3 housing
WITH 4 loan
WITH 5 contact
WITH 6 month
WITH 7 poutcome
1 marital
WITH 0 education
WITH 1 default
WITH 2 housing
WITH 3 loan
WITH 4 contact
WITH 5 month
WITH 6 poutcome
2 education
WITH 0 default
WITH 1 housing
WITH 2 loan
WITH 3 contact
WITH 4 month
WITH 5 poutcome
3 default
WITH 0 housing
WITH 1 loan
WITH 2 contact
WITH 3 month
WITH 4 poutcome
4 housing
WITH 0 loan
WITH 1 contact
WITH 2 month
WITH 3 poutcome
5 loan
WITH 0 contact
WITH 1 month
WITH 2 poutcome
6 contact
WITH 0 month
WITH 1 poutcome
7 month
WITH 0 poutcome
8 poutcome


In [29]:
df_interact.shape

(4521, 53)

In [30]:
df_interact.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,job_marital,job_education,job_default,job_housing,job_loan,job_contact,job_month,job_poutcome,marital_education,marital_default,marital_housing,marital_loan,marital_contact,marital_month,marital_poutcome,education_default,education_housing,education_loan,education_contact,education_month,education_poutcome,default_housing,default_loan,default_contact,default_month,default_poutcome,housing_loan,housing_contact,housing_month,housing_poutcome,loan_contact,loan_month,loan_poutcome,contact_month,contact_poutcome,month_poutcome
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0,unemployed_married,unemployed_primary,unemployed_no,unemployed_no,unemployed_no,unemployed_cellular,unemployed_oct,unemployed_unknown,married_primary,married_no,married_no,married_no,married_cellular,married_oct,married_unknown,primary_no,primary_no,primary_no,primary_cellular,primary_oct,primary_unknown,no_no,no_no,no_cellular,no_oct,no_unknown,no_no,no_cellular,no_oct,no_unknown,no_cellular,no_oct,no_unknown,cellular_oct,cellular_unknown,oct_unknown
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0,services_married,services_secondary,services_no,services_yes,services_yes,services_cellular,services_may,services_failure,married_secondary,married_no,married_yes,married_yes,married_cellular,married_may,married_failure,secondary_no,secondary_yes,secondary_yes,secondary_cellular,secondary_may,secondary_failure,no_yes,no_yes,no_cellular,no_may,no_failure,yes_yes,yes_cellular,yes_may,yes_failure,yes_cellular,yes_may,yes_failure,cellular_may,cellular_failure,may_failure
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0,management_single,management_tertiary,management_no,management_yes,management_no,management_cellular,management_apr,management_failure,single_tertiary,single_no,single_yes,single_no,single_cellular,single_apr,single_failure,tertiary_no,tertiary_yes,tertiary_no,tertiary_cellular,tertiary_apr,tertiary_failure,no_yes,no_no,no_cellular,no_apr,no_failure,yes_no,yes_cellular,yes_apr,yes_failure,no_cellular,no_apr,no_failure,cellular_apr,cellular_failure,apr_failure
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0,management_married,management_tertiary,management_no,management_yes,management_yes,management_unknown,management_jun,management_unknown,married_tertiary,married_no,married_yes,married_yes,married_unknown,married_jun,married_unknown,tertiary_no,tertiary_yes,tertiary_yes,tertiary_unknown,tertiary_jun,tertiary_unknown,no_yes,no_yes,no_unknown,no_jun,no_unknown,yes_yes,yes_unknown,yes_jun,yes_unknown,yes_unknown,yes_jun,yes_unknown,unknown_jun,unknown_unknown,jun_unknown
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0,blue-collar_married,blue-collar_secondary,blue-collar_no,blue-collar_yes,blue-collar_no,blue-collar_unknown,blue-collar_may,blue-collar_unknown,married_secondary,married_no,married_yes,married_no,married_unknown,married_may,married_unknown,secondary_no,secondary_yes,secondary_no,secondary_unknown,secondary_may,secondary_unknown,no_yes,no_no,no_unknown,no_may,no_unknown,yes_no,yes_unknown,yes_may,yes_unknown,no_unknown,no_may,no_unknown,unknown_may,unknown_unknown,may_unknown


## Бинаризация категориальных признаков (dummies, OHE) + попарные взаимодействия
Получилось аж 824 бинарных признака – многовато для такой задачи, и тут случайный лес начинает не справляться, да и логистическая регрессия сработала хуже, чем в прошлый раз.

In [44]:
df_interact_cat_dummies = pd.get_dummies(df_interact, columns=df_interact.columns[df_interact.dtypes == 'object']).drop('y', axis=1)

In [45]:
df_interact_cat_dummies.shape

(4521, 824)

In [46]:
df_interact_cat_dummies_part, df_interact_cat_dummies_valid = train_test_split(df_interact_cat_dummies, test_size=.3, 
                                                    stratify=y, random_state=17)

In [47]:
np.mean(cross_val_score(forest, df_interact_cat_dummies_part, y_train_part, cv=skf, scoring='roc_auc'))

0.7919934473777258

In [48]:
forest.fit(df_interact_cat_dummies_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

In [49]:
roc_auc_score(y_valid, forest.predict_proba(df_interact_cat_dummies_valid)[:, 1])

0.7673039561049553

Случайному лесу уже тяжеловато, когда признаков так много, а вот логистической регрессии – норм.

In [50]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(random_state=17)

In [51]:
np.mean(cross_val_score(logit, df_interact_cat_dummies_part, y_train_part, cv=skf, scoring='roc_auc'))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

0.8743901694037088

In [52]:
logit.fit(df_interact_cat_dummies_part, y_train_part)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
roc_auc_score(y_valid, logit.predict_proba(df_interact_cat_dummies_valid)[:, 1])

0.8807243963363864

## Mean Target
Теперь будем использовать технику кодирования категориальных признаков средним значением целевого признака. Это очень мощная техника, правда, надо умело ее использовать – легко переобучиться. 
Основная идея – для каждого значения категориального признака посчитать среднее значение целевого признака и заменить категориальный признак на посчитанные средние. Правда, считать средние надо на кросс-валидации, а то легко переобучиться. 
Но далее я адресую к видео топ-участников соревнований Kaggle, от них можно узнать про эту технику из первых уст. 
- [Специализация](https://www.coursera.org/specializations/aml) "Advanced Machine Learning" на Coursera, [курс](https://www.coursera.org/learn/competitive-data-science)", How to Win a Data Science Competition: Learn from Top Kagglers", несколько видео посвящено различным способам построяния признаков с задействованием целевого, и как при этом не переобучиться. Рассказывает Дмитрий Алтухов
- [Лекция](https://www.youtube.com/watch?v=g335THJxkto) с презентацией решения конкурса Kaggle BNP paribas, Станислав Семенов

Похожая техника [используется](https://tech.yandex.com/catboost/doc/dg/concepts/algorithm-main-stages_cat-to-numberic-docpage/) и в CatBoost.

Для начала давайте таким образом закодируем исходные категориальные признаки.

In [54]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [55]:
train_df, y = df.copy(), df['y']
train_df_part, valid_df, y_train_part, y_valid = train_test_split(train_df.drop('y', axis=1), y, 
                                                                  test_size=.3, stratify=y, 
                                                                               random_state=17)

In [58]:
y_train_part.mean()

0.11536030341340076

In [59]:
def mean_target_enc(train_df, y_train, valid_df, skf):
    import warnings
    warnings.filterwarnings('ignore')
    
    glob_mean = y_train.mean()
    train_df = pd.concat([train_df, pd.Series(y_train, name='y')], axis=1)
    new_train_df = train_df.copy()
    
    cat_features = train_df.columns[train_df.dtypes == 'object'].tolist()    

    for col in cat_features:
        new_train_df[col + '_mean_target'] = [glob_mean for _ in range(new_train_df.shape[0])]

    for train_idx, valid_idx in skf.split(train_df, y_train):
        train_df_cv, valid_df_cv = train_df.iloc[train_idx, :], train_df.iloc[valid_idx, :]

        for col in cat_features:
            
            means = valid_df_cv[col].map(train_df_cv.groupby(col)['y'].mean())
            valid_df_cv[col + '_mean_target'] = means.fillna(glob_mean)
            
        new_train_df.iloc[valid_idx] = valid_df_cv
    
    new_train_df.drop(cat_features + ['y'], axis=1, inplace=True)
    
    for col in cat_features:
        means = valid_df[col].map(train_df.groupby(col)['y'].mean())
        valid_df[col + '_mean_target'] = means.fillna(glob_mean)
        
    valid_df.drop(train_df.columns[train_df.dtypes == 'object'], axis=1, inplace=True)
    
    return new_train_df, valid_df

In [60]:
train_mean_target_part, valid_mean_target = mean_target_enc(train_df_part, y_train_part, valid_df, skf)

In [64]:
train_mean_target_part.head()

,age,balance,day,duration,campaign,pdays,previous,job_mean_target,marital_mean_target,education_mean_target,default_mean_target,housing_mean_target,loan_mean_target,contact_mean_target,month_mean_target,poutcome_mean_target
482,40,694,4,97,4,196,5,0.112128,0.101030,0.111541,0.115849,0.092254,0.122611,0.139435,0.163934,0.120301
566,26,211,29,168,3,-1,0,0.106383,0.128167,0.142292,0.115570,0.143369,0.124474,0.142503,0.123457,0.089806
2209,59,0,18,247,4,-1,0,0.214286,0.100381,0.103987,0.115169,0.144567,0.122782,0.140405,0.094763,0.091522
725,31,1010,15,385,3,364,9,0.132979,0.104180,0.142292,0.115570,0.093286,0.124474,0.142503,0.074264,0.209091
1025,56,1044,3,353,2,-1,0,0.222222,0.104180,0.103825,0.115570,0.143369,0.124474,0.139665,0.085642,0.089806


In [61]:
np.mean(cross_val_score(forest, train_mean_target_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8439875686594386

In [65]:
forest.fit(train_mean_target_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

In [66]:
roc_auc_score(y_valid, forest.predict_proba(valid_mean_target)[:, 1])

0.8670712440487627

## Mean Target + попарные взаимодействия

In [67]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [68]:
train_df, y = df_interact.drop('y', axis=1).copy(), df_interact['y']
train_df_part, valid_df, y_train_part, y_valid = train_test_split(train_df, y, 
                                                                  test_size=.3, stratify=y, 
                                                                               random_state=17)

In [69]:
train_mean_target_part, valid_mean_target = mean_target_enc(train_df_part, y_train_part, valid_df, skf)

In [70]:
np.mean(cross_val_score(forest, train_mean_target_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8350562884869193

In [71]:
forest.fit(train_mean_target_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

In [72]:
roc_auc_score(y_valid, forest.predict_proba(valid_mean_target)[:, 1])

0.8690167381882619

Опять лучше справляется логистическая регрессия.

In [73]:
np.mean(cross_val_score(logit, train_mean_target_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8915079258110478

In [74]:
logit.fit(train_mean_target_part, y_train_part)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [75]:
roc_auc_score(y_valid, logit.predict_proba(valid_mean_target)[:, 1])

0.9053726595358569

## Catboost
В библиотеке [Catboost](https://catboost.yandex), помимо всего прочего, реализована как раз техника кодирования категориальных значений средним значением целевого признака. Результаты получаются хорошими именно когда в данных много важных категориальных признаков. Из минусов можно отметить меньшую (пока что) производительность в сравнении с Xgboost и LightGBM.

In [76]:
from catboost import CatBoostClassifier

In [77]:
ctb = CatBoostClassifier(random_seed=17)

In [78]:
train_df, y = df.drop('y', axis=1), df['y']
train_df_part, valid_df, y_train_part, y_valid = train_test_split(train_df, y, 
                                                                  test_size=.3, stratify=y, 
                                                                  random_state=17)

In [95]:
train_df_part.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3164 entries, 482 to 4173
Data columns (total 16 columns):
age          3164 non-null int64
job          3164 non-null object
marital      3164 non-null object
education    3164 non-null object
default      3164 non-null object
balance      3164 non-null int64
housing      3164 non-null object
loan         3164 non-null object
contact      3164 non-null object
day          3164 non-null int64
month        3164 non-null object
duration     3164 non-null int64
campaign     3164 non-null int64
pdays        3164 non-null int64
previous     3164 non-null int64
poutcome     3164 non-null object
dtypes: int64(7), object(9)
memory usage: 420.2+ KB


In [79]:
cat_features_idx = np.where(train_df_part.dtypes == 'object')[0].tolist()

In [81]:
train_df_part.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
482,40,technician,married,secondary,no,694,yes,no,cellular,4,feb,97,4,196,5,failure
566,26,self-employed,single,tertiary,no,211,no,no,cellular,29,jan,168,3,-1,0,unknown
2209,59,retired,married,secondary,no,0,no,no,cellular,18,jul,247,4,-1,0,unknown
725,31,management,married,tertiary,no,1010,yes,no,cellular,15,may,385,3,364,9,other
1025,56,retired,married,secondary,no,1044,no,no,telephone,3,jul,353,2,-1,0,unknown


In [80]:
cat_features_idx

[1, 2, 3, 4, 6, 7, 8, 10, 15]

In [82]:
%%time
cv_scores = []
for train_idx, test_idx in skf.split(train_df_part, y_train_part):
    cv_train_df, cv_valid_df = train_df_part.iloc[train_idx, :], train_df_part.iloc[test_idx, :]
    y_cv_train, y_cv_valid = y_train_part.iloc[train_idx], y_train_part.iloc[test_idx]
    
    ctb.fit(cv_train_df, y_cv_train,
        cat_features=cat_features_idx);
    
    cv_scores.append(roc_auc_score(y_cv_valid, ctb.predict_proba(cv_valid_df)[:, 1]))

Learning rate set to 0.02179
0:	learn: 0.6697101	total: 77.5ms	remaining: 1m 17s
1:	learn: 0.6475756	total: 94ms	remaining: 46.9s
2:	learn: 0.6327331	total: 103ms	remaining: 34.1s
3:	learn: 0.6125712	total: 122ms	remaining: 30.4s
4:	learn: 0.5972384	total: 139ms	remaining: 27.7s
5:	learn: 0.5793834	total: 160ms	remaining: 26.4s
6:	learn: 0.5624773	total: 172ms	remaining: 24.4s
7:	learn: 0.5503571	total: 181ms	remaining: 22.5s
8:	learn: 0.5358257	total: 197ms	remaining: 21.7s
9:	learn: 0.5218813	total: 214ms	remaining: 21.2s
10:	learn: 0.5101448	total: 222ms	remaining: 20s
11:	learn: 0.5004410	total: 228ms	remaining: 18.8s
12:	learn: 0.4866016	total: 247ms	remaining: 18.7s
13:	learn: 0.4780427	total: 255ms	remaining: 18s
14:	learn: 0.4691578	total: 263ms	remaining: 17.3s
15:	learn: 0.4591666	total: 272ms	remaining: 16.7s
16:	learn: 0.4493871	total: 292ms	remaining: 16.9s
17:	learn: 0.4388486	total: 304ms	remaining: 16.6s
18:	learn: 0.4296159	total: 324ms	remaining: 16.7s
19:	learn: 0.42

161:	learn: 0.2245866	total: 2.96s	remaining: 15.3s
162:	learn: 0.2243711	total: 2.99s	remaining: 15.3s
163:	learn: 0.2243152	total: 3s	remaining: 15.3s
164:	learn: 0.2240901	total: 3.02s	remaining: 15.3s
165:	learn: 0.2240067	total: 3.04s	remaining: 15.3s
166:	learn: 0.2239649	total: 3.05s	remaining: 15.2s
167:	learn: 0.2238831	total: 3.06s	remaining: 15.2s
168:	learn: 0.2238096	total: 3.07s	remaining: 15.1s
169:	learn: 0.2237452	total: 3.08s	remaining: 15s
170:	learn: 0.2235366	total: 3.1s	remaining: 15s
171:	learn: 0.2234548	total: 3.12s	remaining: 15s
172:	learn: 0.2234021	total: 3.12s	remaining: 14.9s
173:	learn: 0.2232606	total: 3.14s	remaining: 14.9s
174:	learn: 0.2230554	total: 3.15s	remaining: 14.9s
175:	learn: 0.2228509	total: 3.17s	remaining: 14.8s
176:	learn: 0.2227472	total: 3.19s	remaining: 14.9s
177:	learn: 0.2225303	total: 3.22s	remaining: 14.9s
178:	learn: 0.2224975	total: 3.23s	remaining: 14.8s
179:	learn: 0.2224537	total: 3.24s	remaining: 14.8s
180:	learn: 0.2222124	

329:	learn: 0.2068280	total: 5.73s	remaining: 11.6s
330:	learn: 0.2065878	total: 5.75s	remaining: 11.6s
331:	learn: 0.2064039	total: 5.77s	remaining: 11.6s
332:	learn: 0.2063704	total: 5.79s	remaining: 11.6s
333:	learn: 0.2062674	total: 5.81s	remaining: 11.6s
334:	learn: 0.2062094	total: 5.82s	remaining: 11.6s
335:	learn: 0.2061898	total: 5.83s	remaining: 11.5s
336:	learn: 0.2061207	total: 5.84s	remaining: 11.5s
337:	learn: 0.2060259	total: 5.86s	remaining: 11.5s
338:	learn: 0.2059834	total: 5.87s	remaining: 11.5s
339:	learn: 0.2059316	total: 5.88s	remaining: 11.4s
340:	learn: 0.2058016	total: 5.91s	remaining: 11.4s
341:	learn: 0.2057860	total: 5.92s	remaining: 11.4s
342:	learn: 0.2057544	total: 5.93s	remaining: 11.4s
343:	learn: 0.2057413	total: 5.94s	remaining: 11.3s
344:	learn: 0.2057041	total: 5.95s	remaining: 11.3s
345:	learn: 0.2056645	total: 5.96s	remaining: 11.3s
346:	learn: 0.2054840	total: 5.98s	remaining: 11.3s
347:	learn: 0.2052288	total: 6s	remaining: 11.2s
348:	learn: 0.2

488:	learn: 0.1903731	total: 8.88s	remaining: 9.28s
489:	learn: 0.1902892	total: 8.9s	remaining: 9.26s
490:	learn: 0.1902607	total: 8.92s	remaining: 9.25s
491:	learn: 0.1901372	total: 8.94s	remaining: 9.23s
492:	learn: 0.1901149	total: 8.96s	remaining: 9.22s
493:	learn: 0.1900113	total: 8.99s	remaining: 9.21s
494:	learn: 0.1899090	total: 9.01s	remaining: 9.19s
495:	learn: 0.1898813	total: 9.03s	remaining: 9.18s
496:	learn: 0.1898679	total: 9.06s	remaining: 9.17s
497:	learn: 0.1897633	total: 9.08s	remaining: 9.15s
498:	learn: 0.1896690	total: 9.11s	remaining: 9.14s
499:	learn: 0.1896286	total: 9.13s	remaining: 9.13s
500:	learn: 0.1895307	total: 9.15s	remaining: 9.12s
501:	learn: 0.1894903	total: 9.18s	remaining: 9.1s
502:	learn: 0.1894297	total: 9.2s	remaining: 9.09s
503:	learn: 0.1894086	total: 9.22s	remaining: 9.08s
504:	learn: 0.1892852	total: 9.25s	remaining: 9.07s
505:	learn: 0.1892530	total: 9.28s	remaining: 9.06s
506:	learn: 0.1892222	total: 9.3s	remaining: 9.04s
507:	learn: 0.18

651:	learn: 0.1765429	total: 12.7s	remaining: 6.76s
652:	learn: 0.1764438	total: 12.7s	remaining: 6.74s
653:	learn: 0.1763308	total: 12.7s	remaining: 6.72s
654:	learn: 0.1762066	total: 12.7s	remaining: 6.7s
655:	learn: 0.1761157	total: 12.8s	remaining: 6.69s
656:	learn: 0.1760929	total: 12.8s	remaining: 6.67s
657:	learn: 0.1760265	total: 12.8s	remaining: 6.65s
658:	learn: 0.1760130	total: 12.8s	remaining: 6.63s
659:	learn: 0.1759766	total: 12.8s	remaining: 6.62s
660:	learn: 0.1758634	total: 12.9s	remaining: 6.6s
661:	learn: 0.1758349	total: 12.9s	remaining: 6.58s
662:	learn: 0.1756515	total: 12.9s	remaining: 6.56s
663:	learn: 0.1753697	total: 12.9s	remaining: 6.54s
664:	learn: 0.1751571	total: 12.9s	remaining: 6.52s
665:	learn: 0.1750806	total: 13s	remaining: 6.5s
666:	learn: 0.1748543	total: 13s	remaining: 6.49s
667:	learn: 0.1748263	total: 13s	remaining: 6.47s
668:	learn: 0.1747474	total: 13s	remaining: 6.45s
669:	learn: 0.1747129	total: 13.1s	remaining: 6.44s
670:	learn: 0.1746914	t

...

259:	learn: 0.2221958	total: 4.31s	remaining: 12.3s
260:	learn: 0.2221775	total: 4.33s	remaining: 12.2s
261:	learn: 0.2220796	total: 4.34s	remaining: 12.2s
262:	learn: 0.2219577	total: 4.37s	remaining: 12.2s
263:	learn: 0.2218925	total: 4.38s	remaining: 12.2s
264:	learn: 0.2218633	total: 4.39s	remaining: 12.2s
265:	learn: 0.2217751	total: 4.41s	remaining: 12.2s
266:	learn: 0.2217590	total: 4.42s	remaining: 12.1s
267:	learn: 0.2217306	total: 4.43s	remaining: 12.1s
268:	learn: 0.2217112	total: 4.44s	remaining: 12.1s
269:	learn: 0.2216142	total: 4.46s	remaining: 12s
270:	learn: 0.2214309	total: 4.48s	remaining: 12s
271:	learn: 0.2214177	total: 4.49s	remaining: 12s
272:	learn: 0.2213438	total: 4.5s	remaining: 12s
273:	learn: 0.2213248	total: 4.51s	remaining: 11.9s
274:	learn: 0.2213212	total: 4.51s	remaining: 11.9s
275:	learn: 0.2213123	total: 4.52s	remaining: 11.9s
276:	learn: 0.2211314	total: 4.54s	remaining: 11.9s
277:	learn: 0.2209838	total: 4.56s	remaining: 11.9s
278:	learn: 0.2207216

423:	learn: 0.2081770	total: 7.11s	remaining: 9.66s
424:	learn: 0.2080561	total: 7.14s	remaining: 9.66s
425:	learn: 0.2079230	total: 7.16s	remaining: 9.65s
426:	learn: 0.2079180	total: 7.17s	remaining: 9.62s
427:	learn: 0.2077330	total: 7.19s	remaining: 9.61s
428:	learn: 0.2076760	total: 7.21s	remaining: 9.6s
429:	learn: 0.2076228	total: 7.24s	remaining: 9.59s
430:	learn: 0.2075613	total: 7.26s	remaining: 9.58s
431:	learn: 0.2072572	total: 7.28s	remaining: 9.57s
432:	learn: 0.2072295	total: 7.3s	remaining: 9.56s
433:	learn: 0.2071956	total: 7.33s	remaining: 9.56s
434:	learn: 0.2069587	total: 7.35s	remaining: 9.55s
435:	learn: 0.2069354	total: 7.37s	remaining: 9.54s
436:	learn: 0.2066257	total: 7.39s	remaining: 9.53s
437:	learn: 0.2065938	total: 7.41s	remaining: 9.51s
438:	learn: 0.2065630	total: 7.43s	remaining: 9.5s
439:	learn: 0.2064529	total: 7.45s	remaining: 9.49s
440:	learn: 0.2063736	total: 7.48s	remaining: 9.48s
441:	learn: 0.2063424	total: 7.5s	remaining: 9.47s
442:	learn: 0.20

587:	learn: 0.1900075	total: 11.1s	remaining: 7.75s
588:	learn: 0.1898879	total: 11.1s	remaining: 7.73s
589:	learn: 0.1898169	total: 11.1s	remaining: 7.72s
590:	learn: 0.1896731	total: 11.1s	remaining: 7.71s
591:	learn: 0.1895894	total: 11.2s	remaining: 7.69s
592:	learn: 0.1895413	total: 11.2s	remaining: 7.68s
593:	learn: 0.1892133	total: 11.2s	remaining: 7.66s
594:	learn: 0.1891804	total: 11.2s	remaining: 7.65s
595:	learn: 0.1888803	total: 11.3s	remaining: 7.63s
596:	learn: 0.1888773	total: 11.3s	remaining: 7.62s
597:	learn: 0.1886760	total: 11.3s	remaining: 7.6s
598:	learn: 0.1886641	total: 11.3s	remaining: 7.58s
599:	learn: 0.1885084	total: 11.4s	remaining: 7.57s
600:	learn: 0.1884685	total: 11.4s	remaining: 7.55s
601:	learn: 0.1879500	total: 11.4s	remaining: 7.54s
602:	learn: 0.1878714	total: 11.4s	remaining: 7.53s
603:	learn: 0.1878451	total: 11.5s	remaining: 7.51s
604:	learn: 0.1878008	total: 11.5s	remaining: 7.5s
605:	learn: 0.1874786	total: 11.5s	remaining: 7.48s
606:	learn: 0.

748:	learn: 0.1743996	total: 14.9s	remaining: 4.98s
749:	learn: 0.1743550	total: 14.9s	remaining: 4.96s
750:	learn: 0.1743258	total: 14.9s	remaining: 4.94s
751:	learn: 0.1742356	total: 14.9s	remaining: 4.92s
752:	learn: 0.1742260	total: 14.9s	remaining: 4.9s
753:	learn: 0.1742028	total: 15s	remaining: 4.88s
754:	learn: 0.1741700	total: 15s	remaining: 4.86s
755:	learn: 0.1741296	total: 15s	remaining: 4.84s
756:	learn: 0.1740897	total: 15s	remaining: 4.83s
757:	learn: 0.1738998	total: 15.1s	remaining: 4.81s
758:	learn: 0.1738737	total: 15.1s	remaining: 4.79s
759:	learn: 0.1738573	total: 15.1s	remaining: 4.77s
760:	learn: 0.1737808	total: 15.1s	remaining: 4.75s
761:	learn: 0.1737044	total: 15.1s	remaining: 4.73s
762:	learn: 0.1736618	total: 15.2s	remaining: 4.71s
763:	learn: 0.1735097	total: 15.2s	remaining: 4.69s
764:	learn: 0.1733432	total: 15.2s	remaining: 4.67s
765:	learn: 0.1733200	total: 15.2s	remaining: 4.65s
766:	learn: 0.1732578	total: 15.3s	remaining: 4.64s
767:	learn: 0.1731762

914:	learn: 0.1631338	total: 18.6s	remaining: 1.73s
915:	learn: 0.1631050	total: 18.7s	remaining: 1.71s
916:	learn: 0.1630767	total: 18.7s	remaining: 1.69s
917:	learn: 0.1629800	total: 18.7s	remaining: 1.67s
918:	learn: 0.1629234	total: 18.7s	remaining: 1.65s
919:	learn: 0.1629198	total: 18.8s	remaining: 1.63s
920:	learn: 0.1628805	total: 18.8s	remaining: 1.61s
921:	learn: 0.1628711	total: 18.8s	remaining: 1.59s
922:	learn: 0.1628595	total: 18.8s	remaining: 1.57s
923:	learn: 0.1627309	total: 18.8s	remaining: 1.55s
924:	learn: 0.1627148	total: 18.9s	remaining: 1.53s
925:	learn: 0.1625329	total: 18.9s	remaining: 1.51s
926:	learn: 0.1625216	total: 18.9s	remaining: 1.49s
927:	learn: 0.1625084	total: 18.9s	remaining: 1.47s
928:	learn: 0.1624343	total: 19s	remaining: 1.45s
929:	learn: 0.1623387	total: 19s	remaining: 1.43s
930:	learn: 0.1623154	total: 19s	remaining: 1.41s
931:	learn: 0.1623051	total: 19s	remaining: 1.39s
932:	learn: 0.1622784	total: 19s	remaining: 1.37s
933:	learn: 0.1621699	

In [91]:
cat_features_idx

[1, 2, 3, 4, 6, 7, 8, 10, 15]

In [83]:
np.mean(cv_scores)

0.9005784984368927

In [96]:
valid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1357 entries, 3552 to 3904
Data columns (total 16 columns):
age          1357 non-null int64
job          1357 non-null object
marital      1357 non-null object
education    1357 non-null object
default      1357 non-null object
balance      1357 non-null int64
housing      1357 non-null object
loan         1357 non-null object
contact      1357 non-null object
day          1357 non-null int64
month        1357 non-null object
duration     1357 non-null int64
campaign     1357 non-null int64
pdays        1357 non-null int64
previous     1357 non-null int64
poutcome     1357 non-null object
dtypes: int64(7), object(9)
memory usage: 180.2+ KB


In [84]:
%%time
ctb.fit(train_df_part, y_train_part,
        cat_features=cat_features_idx);

Learning rate set to 0.023211
0:	learn: 0.6667391	total: 21ms	remaining: 21s
1:	learn: 0.6423247	total: 45.3ms	remaining: 22.6s
2:	learn: 0.6182561	total: 69.3ms	remaining: 23s
3:	learn: 0.5997096	total: 86.9ms	remaining: 21.6s
4:	learn: 0.5804962	total: 109ms	remaining: 21.6s
5:	learn: 0.5632466	total: 130ms	remaining: 21.6s
6:	learn: 0.5466916	total: 152ms	remaining: 21.6s
7:	learn: 0.5297737	total: 163ms	remaining: 20.2s
8:	learn: 0.5180969	total: 172ms	remaining: 18.9s
9:	learn: 0.5032803	total: 193ms	remaining: 19.1s
10:	learn: 0.4931403	total: 205ms	remaining: 18.4s
11:	learn: 0.4836170	total: 213ms	remaining: 17.6s
12:	learn: 0.4713271	total: 225ms	remaining: 17.1s
13:	learn: 0.4577966	total: 246ms	remaining: 17.3s
14:	learn: 0.4473978	total: 257ms	remaining: 16.9s
15:	learn: 0.4371805	total: 266ms	remaining: 16.4s
16:	learn: 0.4258150	total: 287ms	remaining: 16.6s
17:	learn: 0.4178063	total: 301ms	remaining: 16.4s
18:	learn: 0.4085884	total: 320ms	remaining: 16.5s
19:	learn: 0.

162:	learn: 0.2233140	total: 3.18s	remaining: 16.3s
163:	learn: 0.2232837	total: 3.19s	remaining: 16.3s
164:	learn: 0.2230416	total: 3.22s	remaining: 16.3s
165:	learn: 0.2229150	total: 3.23s	remaining: 16.2s
166:	learn: 0.2225240	total: 3.25s	remaining: 16.2s
167:	learn: 0.2223981	total: 3.27s	remaining: 16.2s
168:	learn: 0.2222308	total: 3.29s	remaining: 16.2s
169:	learn: 0.2219625	total: 3.31s	remaining: 16.2s
170:	learn: 0.2219624	total: 3.32s	remaining: 16.1s
171:	learn: 0.2218992	total: 3.33s	remaining: 16.1s
172:	learn: 0.2217438	total: 3.35s	remaining: 16s
173:	learn: 0.2216293	total: 3.36s	remaining: 16s
174:	learn: 0.2216224	total: 3.37s	remaining: 15.9s
175:	learn: 0.2214485	total: 3.39s	remaining: 15.9s
176:	learn: 0.2208879	total: 3.42s	remaining: 15.9s
177:	learn: 0.2206509	total: 3.44s	remaining: 15.9s
178:	learn: 0.2202049	total: 3.47s	remaining: 15.9s
179:	learn: 0.2199237	total: 3.5s	remaining: 15.9s
180:	learn: 0.2197592	total: 3.52s	remaining: 15.9s
181:	learn: 0.219

322:	learn: 0.2046527	total: 6.24s	remaining: 13.1s
323:	learn: 0.2045948	total: 6.26s	remaining: 13.1s
324:	learn: 0.2043876	total: 6.28s	remaining: 13s
325:	learn: 0.2042819	total: 6.3s	remaining: 13s
326:	learn: 0.2042780	total: 6.32s	remaining: 13s
327:	learn: 0.2042055	total: 6.33s	remaining: 13s
328:	learn: 0.2041370	total: 6.35s	remaining: 12.9s
329:	learn: 0.2040857	total: 6.37s	remaining: 12.9s
330:	learn: 0.2040729	total: 6.38s	remaining: 12.9s
331:	learn: 0.2040529	total: 6.38s	remaining: 12.8s
332:	learn: 0.2040467	total: 6.4s	remaining: 12.8s
333:	learn: 0.2037664	total: 6.43s	remaining: 12.8s
334:	learn: 0.2036997	total: 6.46s	remaining: 12.8s
335:	learn: 0.2036414	total: 6.47s	remaining: 12.8s
336:	learn: 0.2036382	total: 6.49s	remaining: 12.8s
337:	learn: 0.2035211	total: 6.51s	remaining: 12.8s
338:	learn: 0.2033213	total: 6.54s	remaining: 12.8s
339:	learn: 0.2031810	total: 6.57s	remaining: 12.7s
340:	learn: 0.2030370	total: 6.59s	remaining: 12.7s
341:	learn: 0.2028125	

484:	learn: 0.1874339	total: 9.84s	remaining: 10.4s
485:	learn: 0.1873080	total: 9.86s	remaining: 10.4s
486:	learn: 0.1870912	total: 9.88s	remaining: 10.4s
487:	learn: 0.1868699	total: 9.91s	remaining: 10.4s
488:	learn: 0.1867034	total: 9.93s	remaining: 10.4s
489:	learn: 0.1864823	total: 9.95s	remaining: 10.4s
490:	learn: 0.1863406	total: 9.98s	remaining: 10.3s
491:	learn: 0.1862131	total: 10s	remaining: 10.3s
492:	learn: 0.1861239	total: 10s	remaining: 10.3s
493:	learn: 0.1860434	total: 10.1s	remaining: 10.3s
494:	learn: 0.1860103	total: 10.1s	remaining: 10.3s
495:	learn: 0.1859596	total: 10.1s	remaining: 10.3s
496:	learn: 0.1858658	total: 10.1s	remaining: 10.2s
497:	learn: 0.1857290	total: 10.1s	remaining: 10.2s
498:	learn: 0.1854923	total: 10.2s	remaining: 10.2s
499:	learn: 0.1853472	total: 10.2s	remaining: 10.2s
500:	learn: 0.1852840	total: 10.2s	remaining: 10.2s
501:	learn: 0.1851221	total: 10.2s	remaining: 10.2s
502:	learn: 0.1850707	total: 10.3s	remaining: 10.1s
503:	learn: 0.18

650:	learn: 0.1718239	total: 13.8s	remaining: 7.42s
651:	learn: 0.1717469	total: 13.9s	remaining: 7.4s
652:	learn: 0.1717251	total: 13.9s	remaining: 7.38s
653:	learn: 0.1716522	total: 13.9s	remaining: 7.36s
654:	learn: 0.1716200	total: 13.9s	remaining: 7.34s
655:	learn: 0.1715326	total: 14s	remaining: 7.32s
656:	learn: 0.1714700	total: 14s	remaining: 7.3s
657:	learn: 0.1714649	total: 14s	remaining: 7.28s
658:	learn: 0.1714209	total: 14s	remaining: 7.26s
659:	learn: 0.1713506	total: 14.1s	remaining: 7.24s
660:	learn: 0.1712946	total: 14.1s	remaining: 7.22s
661:	learn: 0.1712749	total: 14.1s	remaining: 7.2s
662:	learn: 0.1711841	total: 14.1s	remaining: 7.18s
663:	learn: 0.1711587	total: 14.2s	remaining: 7.16s
664:	learn: 0.1710914	total: 14.2s	remaining: 7.14s
665:	learn: 0.1710338	total: 14.2s	remaining: 7.12s
666:	learn: 0.1708826	total: 14.2s	remaining: 7.1s
667:	learn: 0.1707113	total: 14.3s	remaining: 7.09s
668:	learn: 0.1705861	total: 14.3s	remaining: 7.07s
669:	learn: 0.1705471	to

813:	learn: 0.1623182	total: 17.8s	remaining: 4.07s
814:	learn: 0.1622977	total: 17.9s	remaining: 4.05s
815:	learn: 0.1622601	total: 17.9s	remaining: 4.03s
816:	learn: 0.1622195	total: 17.9s	remaining: 4.01s
817:	learn: 0.1621911	total: 17.9s	remaining: 3.99s
818:	learn: 0.1621414	total: 18s	remaining: 3.97s
819:	learn: 0.1621121	total: 18s	remaining: 3.95s
820:	learn: 0.1619116	total: 18s	remaining: 3.92s
821:	learn: 0.1618838	total: 18s	remaining: 3.9s
822:	learn: 0.1618153	total: 18.1s	remaining: 3.88s
823:	learn: 0.1617712	total: 18.1s	remaining: 3.86s
824:	learn: 0.1617411	total: 18.1s	remaining: 3.84s
825:	learn: 0.1616476	total: 18.1s	remaining: 3.82s
826:	learn: 0.1616230	total: 18.2s	remaining: 3.8s
827:	learn: 0.1615287	total: 18.2s	remaining: 3.78s
828:	learn: 0.1614147	total: 18.2s	remaining: 3.75s
829:	learn: 0.1612639	total: 18.2s	remaining: 3.73s
830:	learn: 0.1612382	total: 18.3s	remaining: 3.71s
831:	learn: 0.1610252	total: 18.3s	remaining: 3.69s
832:	learn: 0.1609003	

979:	learn: 0.1522211	total: 21.8s	remaining: 445ms
980:	learn: 0.1522126	total: 21.8s	remaining: 423ms
981:	learn: 0.1522043	total: 21.9s	remaining: 401ms
982:	learn: 0.1521835	total: 21.9s	remaining: 378ms
983:	learn: 0.1520898	total: 21.9s	remaining: 356ms
984:	learn: 0.1520810	total: 21.9s	remaining: 334ms
985:	learn: 0.1520752	total: 22s	remaining: 312ms
986:	learn: 0.1520296	total: 22s	remaining: 289ms
987:	learn: 0.1520048	total: 22s	remaining: 267ms
988:	learn: 0.1519464	total: 22s	remaining: 245ms
989:	learn: 0.1519367	total: 22.1s	remaining: 223ms
990:	learn: 0.1518770	total: 22.1s	remaining: 200ms
991:	learn: 0.1518614	total: 22.1s	remaining: 178ms
992:	learn: 0.1518494	total: 22.1s	remaining: 156ms
993:	learn: 0.1518071	total: 22.1s	remaining: 134ms
994:	learn: 0.1517889	total: 22.2s	remaining: 111ms
995:	learn: 0.1516644	total: 22.2s	remaining: 89.1ms
996:	learn: 0.1516565	total: 22.2s	remaining: 66.8ms
997:	learn: 0.1516168	total: 22.2s	remaining: 44.6ms
998:	learn: 0.151

In [85]:
roc_auc_score(y_valid, ctb.predict_proba(valid_df)[:, 1])

0.9171577104549627